# Semantic Kernel + Hugging Faces Embedding

This example shows how to configure Semantic Kernel with Hugging Faces.

It is based on the [Example 06 of memory and embeddings](https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/06-memory-and-embeddings.ipynb) of Semantic Kernel

**Step 1**: Install Semantic Kernel 

In [1]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Connectors.HuggingFace, 1.7.1-preview"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.7.1-alpha"
#r "nuget: System.Linq.Async, 6.0.1"

Installed Packages Microsoft.SemanticKernel, 1.7.1 Microsoft.SemanticKernel.Connectors.HuggingFace, 1.7.1-preview Microsoft.SemanticKernel.Plugins.Memory, 1.7.1-alpha System.Linq.Async, 6.0.1

**Step 2**: Instantiate the Kernel

In [17]:
using System.Linq;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.HuggingFace.TextGeneration;

// Memory functionality is experimental
#pragma warning disable SKEXP0001, SKEXP0010, SKEXP0050

// HuggingFace functionality is experimental
#pragma warning disable SKEXP0020, SKEXP0070

var kernelBuilder = Kernel.CreateBuilder();

kernelBuilder.AddHuggingFaceTextEmbeddingGeneration(
    model: "sentence-transformers/all-MiniLM-L6-v2",
    endpoint: new Uri("http://127.0.0.1:8080")
);

var kernel = kernelBuilder.Build();

var memoryBuilder = new MemoryBuilder();
memoryBuilder.WithTextEmbeddingGeneration(kernel.GetRequiredService<ITextEmbeddingGenerationService>());
memoryBuilder.WithMemoryStore(new VolatileMemoryStore());
var memory = memoryBuilder.Build();

### Manually Adding Memories

Creating some memories into the volatile store by using `SaveInformationAsync`:


In [18]:
const string MemoryCollectionName = "aboutMe";

await memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "My name is Andrea");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info2", text: "I currently work as a tourist operator");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info3", text: "I currently live in Seattle and have been living there since 2005");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info4", text: "I visited France and Italy five times since 2015");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info5", text: "My family is from New York");


Searching the memory:

In [19]:
var questions = new[]
{
    "what is my name?",
    "where do I live?",
    "I live in",
    "where is my family from?",
    "where have I travelled?",
    "what do I do for work?",
};

foreach (var q in questions)
{
    
    var response = await memory.SearchAsync(MemoryCollectionName, q, minRelevanceScore: 0.5).FirstOrDefaultAsync();

    if (response == null)
    {
        Console.WriteLine(q + " " + "Not found");
    }
    else
    {
        Console.WriteLine(q + " " + response?.Metadata.Text);
    }
}

what is my name? My name is Andrea
where do I live? Not found
I live in Not found
where is my family from? My family is from New York
where have I travelled? Not found
what do I do for work? Not found


I've been trying with different parameters of `minRelevanceScore` but with ADA I get better results.